# Convección de calor forzada: Método implícito

<table border="1">
  <tbody>
    <tr>
      <td><img src="sol0.png" width="100"></td>
      <td><img src="sol1.png" width="100"></td>
    </tr>
    <tr>
      <td><img src="sol2.png" width="100"></td>
      <td><img src="sol3.png" width="100"></td>
    </tr>
  </tbody>
</table>

Aproximar la solución al siguiente problema usando diferencias finitas:

---
## Modelo matemático.

### Ecuación a resolver.
$$
\rho \dfrac{\partial T}{\partial t} + \nabla \cdot\left( \vec{u} T - \kappa \nabla T\right) = 0
$$

El dominio espacial de solución es $(x,y) \in [0,1] \times [0,1]$, mientras que el tiempo varía como sigue $t = 0, \dots, Nt$.

Use $\kappa = 1$ y $\rho = 1$.

---

In [ ]:
# Importar bibliotecas
import numpy as np
from fdm import plot_mesh, plot_contour, plot_vector, Laplaciano2D_NS, RHS_NS

# Definir parámetros físicos y numéricos.
Nx = 21
Ny = 21
Lx = 1.0
Ly = 1.0
hx = Lx / (Nx+1)
hy = Ly / (Ny+1)
h = hx
k = 1.0
print(hx, hy)
print(Nx, Ny)

### Condiciones de frontera.
- $T(0,y) = 0$ 
- $T(1,y) = 0$ 
- $T(x,0) = 0$ 
- $T(x,1) = 0$ 
    

In [ ]:
# Defina la malla

# Número total de nodos incluyendo las fronteras.
NxT = Nx + 2
NyT = Ny + 2

print(NxT, NyT)

# Coordenadas de la malla
xn = np.linspace(0,Lx,NxT)
yn = np.linspace(0,Ly,NyT)
xg, yg = np.meshgrid(xn, yn, indexing='ij', sparse=False)
plot_mesh(Lx, Ly, xg, yg)

In [ ]:
# Defina las condiciones de frontera sobre esa malla.

# Campo escalar sobre la malla
T = np.zeros((NxT, NyT))

# Valores en las fronteras del dominio
TL = 0.0; TR = 0.0
TB = 0.0; TT = 0.0

T[0 , :] = TL # LEFT
T[-1, :] = TR # RIGHT
T[: , 0] = TB # BOTTOM
T[: ,-1] = TT # TOP

plot_contour(Lx, Ly, xg, yg, T, ticks = [0, 1], cmap='viridis', yshared=True)

### Condición inicial.

$$
T(x,y) = \begin{cases}
\sin^2(\frac{\pi}{2}(1-\frac{r}{0.1})) & \text{Para} \,\,\, r < 0.1  \\
0 & \text{Para} \,\,\, r \geq 0.1  \\
\end{cases}
$$

con $r = \sqrt{(x-c_x)^2 + (y-c_y^2)}$ y $c = (c_x,c_y) \in (0,1)\times(0,1)$

In [ ]:
# Definir la condición inicial

cx = 0.25
cy = 0.5

for i in range(0, NxT):
    for j in range(0, NyT):
        r = np.sqrt( (xn[i] - cx)**2 + (yn[j] - cy)**2)
        if r < 0.1:
            T[i,j] = np.sin(0.5 * np.pi * (1 - r / 0.1) )**2
        
plot_contour(Lx, Ly, xg, yg, T, ticks = [0, 1], cmap='viridis', yshared=True)

### Velocidad.
La velocidad está dada por:

$$
u = -A \cos(\alpha_y \pi y) \sin(\alpha_x \pi x) \\
v = A \sin(\alpha_y \pi y) \cos(\alpha_x \pi x)
$$

Utilice  $A = 1.0$ y $\alpha_x = \alpha_y = 2$.

In [ ]:
# Definir la velocidad
v = np.zeros((2, NxT, NyT))
A = 1.0
alpha_x = 2.0
alpha_y = 2.0
v[0] = -A * np.cos(np.pi * alpha_y * yg / Ly) * np.sin(np.pi * alpha_x * xg / Lx)
v[1] =  A * np.sin(np.pi * alpha_y * yg / Ly) * np.cos(np.pi * alpha_x * xg / Lx)

plot_vector(Lx, Ly, xg, yg, v)

In [ ]:
%%time
# Método de Euler hacia atrás (implicito)
ht = 0.01
r = k * ht / h**2
T_new = T.copy()
tolerancia = 1.0e-3 #1.0e-3
error = 1.0
error_lista = []

A = Laplaciano2D_NS(Nx, Ny, r)

count = 1
while(error > tolerancia):
    f = RHS_NS(Nx, Ny, T, r)
    Tt = np.linalg.solve(A,f).reshape(Ny, Nx)
    T_new[1:Nx+1,1:Ny+1] = Tt.T

    error = np.linalg.norm(T_new - T)
    print(count, error)
    error_lista.append(error)
    T[:] = T_new[:]
    count += 1

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12,5))
plt.subplot(1,3,1)
plot_mesh(Lx, Ly, xg, yg)

plt.subplot(1,3,2)
plot_contour(Lx, Ly, xg, yg, T, ticks = [0, .01, 0.], 
             mesh = True, lines = 0, colors = 'white',
             cmap='viridis', yshared=True)
